In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import shapefile as shp
from shapely.geometry import Point
sns.set_style('whitegrid')

In [4]:
fp = r'/home/t-sahuja/cultural_artifacts/notebooks/Maps_with_python-master/india-polygon.shp'
map_df = gpd.read_file(fp) 
map_df_copy = gpd.read_file(fp)
map_df.head()

,id,st_nm,geometry
0,NaN,Andaman and Nicobar Islands,"MULTIPOLYGON (((93.84831 7.24028, 93.92705 7.0..."
1,NaN,Arunachal Pradesh,"POLYGON ((95.23643 26.68105, 95.19594 27.03612..."
2,NaN,Assam,"POLYGON ((95.19594 27.03612, 95.08795 26.94578..."
3,NaN,Bihar,"POLYGON ((88.11357 26.54028, 88.28006 26.37640..."
4,NaN,Chandigarh,"POLYGON ((76.84208 30.76124, 76.83758 30.72552..."


In [9]:
map_df['st_nm']

0     Andaman and Nicobar Islands
1               Arunachal Pradesh
2                           Assam
3                           Bihar
4                      Chandigarh
5                    Chhattisgarh
6          Dadra and Nagar Haveli
7                   Daman and Diu
8                             Goa
9                         Gujarat
10                        Haryana
11               Himachal Pradesh
12                      Jharkhand
13                      Karnataka
14                         Kerala
15                    Lakshadweep
16                 Madhya Pradesh
17                    Maharashtra
18                        Manipur
19                      Meghalaya
20                        Mizoram
21                       Nagaland
22                          Delhi
23                     Puducherry
24                         Punjab
25                      Rajasthan
26                         Sikkim
27                     Tamil Nadu
28                      Telangana
29            

In [6]:
results = pd.read_csv('/home/t-sahuja/cultural_artifacts/notebooks/results_plot.csv')

In [8]:
results.columns

Index(['States', 'Llama 2', 'Falcon', 'GPT-4', 'Palm 2'], dtype='object')

In [12]:
list_of_results = []
li_models = ['Llama 2', 'Falcon', 'GPT-4', 'Palm 2']
for model in li_models:
    score_data = {
        'State': results['States'],
        'Score': results[model]
    }
    list_of_results.append(pd.DataFrame(score_data))

In [13]:
list_of_results[0]

,State,Score
0,Andhra Pradesh,0.07
1,Assam,0.18
2,Bihar,0.50
3,Chattisgarh,0.11
4,Delhi,0.30
5,Gujarat,0.22
6,Jharkhand,0.33
7,Karnataka,0.21
8,Kerala,0.62
9,Maharashtra,0.62


Bad pipe message: %s [b'\x01F\x8d\x83\xe2\xf0WV}\x81Y\x01=\xddK\xe1gz \xb7=8u\xc2\x83\xc0Wb\xdfg\x8b}\xdb\x0e\xdd\xd03`9\xf9&Y?8\xf3\xc7\\\x94\xe2?\x8e\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xc2A\xf6\xf8\xe4\xe0d\xa4\x9ffN\xd5\x87\xa4']
Bad pipe message: %s [b"nF\x9d/c\xc4\x9c\x1b\x9al*\xa6\xc6p\x1d\x8e\x87^ ~\xafR\xdd\xdb\xeb\x1av\xd4LTY'D\xd7\x8c\xf77\xe1N\xa4\x12\xb4\xb8\xcd\xd2\x86\x19|\x86\xa2\xcd\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x0

In [11]:
merged_llama = map_df.set_index('st_nm').join()

{'State': 0     Andhra Pradesh
 1              Assam
 2              Bihar
 3        Chattisgarh
 4              Delhi
 5            Gujarat
 6          Jharkhand
 7          Karnataka
 8             Kerala
 9        Maharashtra
 10            Odisha
 11            Punjab
 12        Tamil Nadu
 13         Telangana
 14       West Bengal
 Name: States, dtype: object,
 'Score': 0     0.07
 1     0.18
 2     0.50
 3     0.11
 4     0.30
 5     0.22
 6     0.33
 7     0.21
 8     0.62
 9     0.62
 10    0.25
 11    0.50
 12    0.55
 13    0.46
 14    0.40
 Name: Llama 2, dtype: float64}